In [1]:
import sqlite3
import random
from pyvis.network import Network
import networkx as nx
from functools import reduce

In [2]:
iVaccInfProb = 0
HexInfColor = '#fc0303'
# 第一個被感染的人
iFirstInfPid = 0

listMaxConn = [1, 4, 4, 10, 10, 10]
dictInfectivity = {1: 0, 2: 0, 3: 0, 4: 0.6, 5: 0.7, 6: 0.6, 7: 0.4, 8: 0.2, 9: 0.1, 10: 0}
iDays = 7
iPopCnt = 500
listPR = []

In [3]:
class Person:
    def __init__(self, iPid, boolInfFlag=False, floatInfProbability=0, boolIfVacc=False, iInfDays=0):
        self.pid = iPid
        self.boolIfInf = boolInfFlag
        self.boolIfVacc = boolIfVacc
        self.floatInfProbability = floatInfProbability
        self.iInfDays=iInfDays
        
    def IsInfected(self):
        self.boolIfInf = True
        
    def IsVacced(self):
        self.boolIfInf = False
        self.boolIfVacc = True
        self.floatInfProbability = iVaccInfProb
        
    def ResetInfStatus(self):
        self.boolIfInf = False
        self.boolIfVacc=False
        self.floatInfProbability = 0
        self.iInfDays = 0

In [4]:
connPR = sqlite3.connect('./PeopleRelation.db')
cur = connPR.cursor()

In [5]:
# drop 所有表格
cur.execute('drop table if exists Relation')
cur.execute('drop table if exists People')

In [6]:
cur.execute(
'''
    create table if not exists People( pid integer not null primary key )
''')

cur.execute(
'''
    create table if not exists Relation( pid integer not null,
    pidconn integer not null,
    primary key(pid,
    pidconn),
    foreign key (pid) references People(pid),
    foreign key (pidconn) references People(pid) )
''')

In [7]:
listPeople = [Person(x) for x in range(iPopCnt)]

In [8]:
# insert data into table People
for person in listPeople:
    cur.execute(f'insert into People values({person.pid})')
connPR.commit()

In [9]:
# 模擬人與人之間的接觸關係
listPR = []
for person in listPeople:
    iRandconnCnt = random.choice(listMaxConn)
    listTmpPidConnTo = random.choices(listPeople, k=iRandconnCnt)
    listPidConnTo = list(filter(lambda x: x.pid != person.pid, listTmpPidConnTo))
    for PersonConnTo in listPidConnTo:
        if person.pid < PersonConnTo.pid:
            tuplePR = (person.pid, PersonConnTo.pid)
        else:
            tuplePR = (PersonConnTo.pid, person.pid)
        listPR.append(tuplePR)
listPR = list(set(listPR))

In [10]:
cur.executemany('insert into Relation values(?, ?)', listPR)
connPR.commit()

In [11]:
netPR = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

for person in listPeople:
    netPR.add_node(person.pid, title=str(person.pid), value=10)

for relation in listPR:
    netPR.add_edge(relation[0], relation[1])

netPR.barnes_hut()
netPR.show('mygraph.html')

In [14]:
netPRInf = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')

for person in listPeople:
    person.ResetInfStatus()

listPeople[iFirstInfPid].IsInfected()

for day in range(iDays):
    for person in listPeople:
        listRelationsWithThisPerson = list(filter(lambda x: x[0] == person.pid or x[1] == person.pid, listPR))
        if listRelationsWithThisPerson == []:
            continue
        # 挑出跟這個人有關的人際關係
        listMightBeInfPids = list(set(reduce(lambda x, y: x + y, listRelationsWithThisPerson)))
        # 移除這個人
        listMightBeInfPids.remove(person.pid)
        # 如果被感染就感染天數開始加一，這個天數會影響感染機率
        if person.boolIfInf == True:
            person.iInfDays += 1
        # 如果感染機率大於 0 ，就開始感染過程。
        # 首先依照被感染天數決定感染機率
        person.floatInfProbability = dictInfectivity.get(person.iInfDays, 0)
        print(person.pid, person.boolIfInf, person.iInfDays, person.floatInfProbability)

        if person.floatInfProbability > 0:
            # 根據這個人的感染機率隨機挑人
            # todo 找出為什麼 random.choices 會挑到重複的元素
            listBeInfPids = random.choices(listMightBeInfPids, k=round(len(listMightBeInfPids) * person.floatInfProbability))
            for iInfPid in listBeInfPids:
                listPeople[iInfPid].IsInfected()
                    
for person in listPeople:
    if person.boolIfInf == True:
        netPRInf.add_node(person.pid, title=str(person.pid), color=HexInfColor, value=10)
    else:
        netPRInf.add_node(person.pid, title=str(person.pid), value=10)

for relation in listPR:
    netPRInf.add_edge(relation[0], relation[1])

print(len(listPeople), len(listPR))

netPRInf.barnes_hut()
netPRInf.show('InfGraph.html')

0 True 13 0
1 False 0 0
2 True 5 0.7
3 False 0 0
4 False 0 0
5 False 0 0
6 True 4 0.6
7 True 1 0
8 False 0 0
9 True 3 0
10 True 3 0
11 False 0 0
12 True 1 0
13 False 0 0
14 False 0 0
15 False 0 0
16 True 1 0
17 False 0 0
18 True 1 0
19 False 0 0
20 False 0 0
21 True 3 0
22 True 2 0
23 True 1 0
24 True 1 0
25 True 12 0
26 False 0 0
27 False 0 0
28 True 2 0
29 True 1 0
30 True 1 0
31 False 0 0
32 True 1 0
33 False 0 0
34 False 0 0
35 True 2 0
36 True 11 0
37 True 2 0
38 True 4 0.6
39 True 3 0
40 False 0 0
41 True 2 0
42 False 0 0
43 True 3 0
44 False 0 0
45 False 0 0
46 True 1 0
47 True 2 0
48 False 0 0
49 False 0 0
50 False 0 0
51 False 0 0
52 False 0 0
53 False 0 0
54 True 3 0
55 False 0 0
56 True 1 0
57 False 0 0
58 False 0 0
59 False 0 0
60 False 0 0
61 False 0 0
62 False 0 0
63 True 5 0.7
64 False 0 0
65 False 0 0
66 False 0 0
67 True 4 0.6
68 True 2 0
69 True 3 0
70 True 1 0
71 True 3 0
72 True 3 0
73 True 3 0
74 False 0 0
75 False 0 0
76 True 1 0
77 True 6 0.6
78 False 0 0
79 True

In [13]:
connPR.close()